# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [1]:
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


## Откройте и изучите файл

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head())
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Так как отдельной тестовой выборки нет, то мы делим исходный датафрейм на 3 выборки (тренировочную, валидационную и тестовую) в соотношении 3:1:1

## Разбейте данные на выборки

In [3]:
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=12345)
train_df, test_df = train_test_split(train_df, test_size=0.25, random_state=12345)

features_train = train_df.drop(['is_ultra'], axis=1)
target_train = train_df['is_ultra']
features_valid = valid_df.drop(['is_ultra'], axis=1)
target_valid = valid_df['is_ultra']
features_test = test_df.drop(['is_ultra'], axis=1)
target_test = test_df['is_ultra']

print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)

(1928, 5)
(643, 5)
(643, 5)


Успешно разбиваем выборки и приступаем к построению моделей

## Исследуйте модели

In [4]:
best_model = None
best_result = 0
best_depth = 0

for depth in range(1,100):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if ((result > 0.75) and model.score(features_test, target_test) > best_result):
        best_model = model
        best_result = model.score(features_test, target_test)
        best_result_valid = result
        best_depth = depth
        

print('Accuracy score для модели "Дерево решений" на валидационной выборке:', best_result_valid, ',\nна тестовой выборке:', \
      best_result, 'с макс длиной:', best_depth)


Accuracy score для модели "Дерево решений" на валидационной выборке: 0.7884914463452566 ,
на тестовой выборке: 0.7744945567651633 с макс длиной: 7


Модель "Дерево решений" имеет неплохие показатели accuracy score на тестовой выборке, но можно заметить, что accuracy score для валидационной выборки выше, скорее всего это связано со спецификой моделей данного типа. Они часто переобучаются и недообучаются. 

In [5]:
best_model = None
best_result = 0
best_est = 0

for est in range(1,25):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if ((result > 0.75) and model.score(features_test, target_test) > best_result):
        best_model = model
        best_result = model.score(features_test, target_test)
        best_result_valid = result
        best_est = est

print('Accuracy score для модели "Случайный лес" на валидационной выборке:', best_result_valid, ',\nна тестовой выборке:', \
      best_result, 'с макс длиной:', best_est)

Accuracy score для модели "Случайный лес" на валидационной выборке: 0.7838258164852255 ,
на тестовой выборке: 0.7931570762052877 с макс длиной: 22


" Accuracy score для модели "Случайный лес" на валидационной выборке: 0.7900466562986003 ,
на тестовой выборке: 0.7993779160186625 с макс длиной: 65 " - такой результат можно получить при n_estimators = 65, но процесс вычисления результата занимает сильно больше времени по сравнению с n_estimators = 22, когда значение accuracy_score мы получем очень близкое: 0.7931570762052877. Разница в 6% может быть и существенна, но зато мы сильно снижаем время подсчета


In [6]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)

print('Accuracy score для модели "Логистическая регрессия" на валидационной выборке:', model.score(features_valid, target_valid), ',\nна тестовой выборке:', \
      model.score(features_test, target_test))

Accuracy score для модели "Логистическая регрессия" на валидационной выборке: 0.7589424572317263 ,
на тестовой выборке: 0.7262830482115086


Модель типа "логистическая регрессия" имеет accuracy score на тестовой выборке менее 75%, что означает, что она нам не подходит.

По результатм исследования, лучшей моделью для данного задания оказался "случайный лес" с показателем accuracy score = 0.7931570762052877

In [7]:
best_model = RandomForestClassifier(random_state=12345, n_estimators=22)
best_model.fit(features_train, target_train)

RandomForestClassifier(n_estimators=22, random_state=12345)

## Проверка модели на адекватность

Для выполенния проверки модели на вменяемость, создадим константную модель и сравним ее показатель accuracy score с показателем для нашей модели. Если результат будет положительный, то наша модель будет считаться вменяемойб а если нет, то бесполезной.

In [8]:
target_allzero_predicion_constant = pd.Series([0] * len(target_test))
print('Разница между качеством лучшей из обученных моделей и константной моделью =', best_model.score(features_test, target_test)-accuracy_score(target_allzero_predicion_constant, target_test))

Разница между качеством лучшей из обученных моделей и константной моделью = 0.10419906687402802


Результат положительный, следовательно, обученная модель вменяема (адекватна)